In [ ]:
import keras 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from thursday import get_data, augment_data
from thursday.models import SklearnModel, HOGNet

In [ ]:
# Path to data output by fri_frii_download.ipynb.
fr_data_path = "data/data.h5"
# Path to Aniyan+17 validation set.
aniyan_path = "data/asu.tsv"

seed = 0

In [ ]:
# Converting the data from data.h5 and asu.tsv into training and testing data.
# Coordinates from data.h5 and asu.tsv are converted into Skycoords.
# The two are cross-matched. Sources within five arcsec radius of each other are assumed to be the same object.
# asu sources are extracted from the data.h5 sources, and formated into class-wise balanced training set.
# The remainder are formated into class-wise balanced testing set.

train_i, test_i = get_data(fr_data_path, aniyan_path, seed=seed)
# TODO(josh-marsh): Add scale

In [ ]:
# Consrtuct the data generator to randomly augment data
datagen = augment_data(dim=256, rotation_range=180, zoom_range=0.2, shift_range=0.0, flip=True)

In [ ]:
# Instantiate Logistic Regression (any other Sklearn Classifier can be used - with the same input data format)
logistic = SklearnModel(LogisticRegression, datagen=datagen, max_augmentations=100, seed=seed,
                        solver='liblinear', max_iter=100, dual=True)

In [ ]:
with h5py.File(fr_data_path, 'r') as data:
    train_x = data['images'][train_i]
    train_y = data['labels'][train_i]
    
    train_x = train_x.reshape((train_x.shape[0], -1))
    
    logistic.fit(train_x, train_y)

In [ ]:
# Save model
logistic.save(out_path)

In [ ]:
with h5py.File(fr_data_path, 'r') as data:
    test_x = data['images'][test_i]
    test_y = data['labels'][test_i]
    
    test_x = test_x.reshape((test_x.shape[0], -1))
    
    predictions = logistic.predict(test_x)
    correct = test_y == predictions
    print('Accuracy: {:.02%}'.format(correct.mean()))